## LSTM para generar cartas de hearthstone

En esta libreta se encuentra la implementación de redes recurrentes lstm con keras, en esta se resuelve el problema de generación de texto a nivel caracter para generar cartas de hearthstone. Este código está basado en la siguiente [entrada](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) de blog escrita por Jason Brownle.

### Dataset a utilizar

Para esta entrada nos interesa entrenar con todas las cartas de hearthstone existentes, para ello nuestros amigos de [HeathSim](https://hearthsim.info/) se tomaron la molestia de crear un archivo json que contiene todas las cartas en todos los idiomas. Para descargarse se puede hacer desde [este enlace](https://hearthstonejson.com/), en este tutorial estaremos usando las cartas en español, pero se puede, utilizar el json del lenguaje que sea.

### La implementación

Ahora nos centraremos en como se implementan las LSTM con keras, para ello primero que nada se necesitan importar los siguientes módulos.

In [1]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import json
from bs4 import BeautifulSoup

C:\Users\jrse9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Ahora leemos el archivo cards.json previamente descargado con la librería json de la librería estandar de python.

In [2]:
with open('cards.json', encoding="utf8") as f:
    data = json.load(f)

Una vez leido el json tenemos que limpiar un poco los textos de las descripciones de las cartas. Para ello se define la lista de caracteres que se quiere remover y se reemplazan con una cadena vacía cada uno. También se pegan los textos de todas las cartas en un string muy largo para trabajar sobre este.

El arreglo char_to_in es un diccionario que asigna un número a cada caracter posible, este será utilizado más adelante.

In [3]:
remove_chars = ["@", ",", ".", "$", ":", "\n", "!", "#", """'""", ";", "?", "[", "]", "{", "}", "|", "\xa0", "¡", '«', '»', "¿", "…"]

raw_text = "" 
for card in data:
    if("text" in card and len(card["text"]) > 1):
        text = card["text"]
        for char in remove_chars:
            text = text.replace(char,"")
        soup = BeautifulSoup(text)
        text = soup.get_text()
        if(len(text) > 1):
            raw_text = raw_text + " " + text

chars = sorted(list(set(raw_text))) # Arreglo de todos los posibles caracteres contenidos en todas las cartas.
char_to_int = dict((c, i) for i, c in enumerate(chars))

Después se calculan el número de caracteres total en el texto y el vocabulario total

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Número total de caracteres: ", n_chars)
print("Vocabulario total: ", n_vocab)

Número total de caracteres:  221457
Vocabulario total:  76


Ahora se generan secuencias de caracteres con longitud de 40, se escogió el 40 porque es la longitud promedio de las cadenas de caracteres de los textos de las cartas. Esta longitud es la profundidad que tendrá la red LSTM para hacer el back propagation a través del tiempo.

In [5]:
seq_length = 40
dataX = [] # "conjunto de entrenamiento"
dataY = [] # "Caracter que sigue despues de los 40"
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Patrones totales: ", n_patterns)

Patrones totales:  221417


Ahora se le aplica un reshape a dataX para que tenga la forma adecuada para el entrenamiento. También se utiliza el método to_categorical de np_utils para transformar las y a one hot(dummy).

In [6]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

X = X / float(n_vocab)

y = np_utils.to_categorical(dataY)

Ahora se define el modelo que se utilizará para entrenar la red, básicamente es una red neuronal secuencial con dos capas LSTM, ambas con dropout de 0.2. Activación softmax para la salida y como optimizador el método de adam.

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Se guardan los checkpoints en la carpeta checkpoints en la raíz del proyecto.

In [8]:
filepath="./checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Se entrena el modelo, la red fue entrenada originalmente por 150 epochs, con un tamaño de batch de 128 y en un GPU NVIDIA geforce gtx 960 de 4gb.

In [9]:
model.fit(X, y, epochs=150, batch_size=128, callbacks=callbacks_list)

Epoch 1/150
221417/221417 [==============================] - 255s 1ms/step - loss: 2.6458

Epoch 00001: loss improved from inf to 2.64577, saving model to ./checkpoints/weights-improvement-01-2.6458.hdf5


OSError: Unable to create file (unable to open file: name = './checkpoints/weights-improvement-01-2.6458.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

### Generar cartas con el modelo ya entrenado

Primero se carga el checkpoint guardado anteriormente durante el entrenamiento.

In [10]:
filename = "./checkpoints/weights-improvement-150-0.4161.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Se define este diccionario para transformar la salida a caracter.

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Se toma una semilla de dataX y apartir de ella se generan 1000 caracteres.

In [83]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Semilla:
" nio 22 Añade El Segundo Sello a tu mano  "
Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final de tu turnorestaura 4 p de saluda tu héroe xCoito de batalla Invoca a un esbirro aleatorio de coste 6 xGl final